In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_54.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_106_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_42.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_81.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_107.wav
/kaggle/input/shl-in

In [2]:
train_df = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/train.csv')
test_df = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment-2025/dataset/csvs/test.csv')
train_df.head(5)

,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [3]:
import os

train_audio_dir = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train"
print(os.listdir(train_audio_dir)[:20])


['audio_49.wav', 'audio_77_2.wav', 'audio_90.wav', 'audio_77.wav', 'audio_66.wav', 'audio_54.wav', 'audio_237.wav', 'audio_42.wav', 'audio_257.wav', 'audio_256_2.wav', 'audio_81.wav', 'audio_330.wav', 'audio_72.wav', 'audio_232_2.wav', 'audio_328.wav', 'audio_107.wav', 'audio_79.wav', 'audio_73_2.wav', 'audio_83.wav', 'audio_88.wav']


In [4]:
def get_audio_path(filename):
    if not filename.endswith(".wav"):
        filename = filename + ".wav"
    return os.path.join(
        "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train",
        filename
    )


In [5]:
import librosa
import numpy as np
import os
from tqdm import tqdm

def extract_audio_features(file_path):
    y, sr = librosa.load(file_path, sr=16000)

    features = []

    # MFCCs (13 x mean + std = 26)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(13):
        features.append(float(np.mean(mfccs[i])))
        features.append(float(np.std(mfccs[i])))

    # Spectral features
    features.append(float(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))))
    features.append(float(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))))
    features.append(float(np.mean(librosa.feature.rms(y=y))))
    features.append(float(np.mean(librosa.feature.zero_crossing_rate(y))))

    # Duration
    features.append(float(len(y) / sr))

    # Tempo (FORCE scalar)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    features.append(float(np.mean(tempo)))

    return np.array(features, dtype=np.float32)



X = []
y = []

AUDIO_DIR = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train"

for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    filename = row["filename"]

    if not filename.endswith(".wav"):
        filename = filename + ".wav"

    audio_path = os.path.join(AUDIO_DIR, filename)

    if not os.path.exists(audio_path):
        print(f"❌ File missing: {audio_path}")
        continue

    try:
        feats = extract_audio_features(audio_path)
        X.append(feats)
        y.append(row["label"])
    except Exception as e:
        print(f"❌ Failed {filename}: {e}")

X = np.array(X)
y = np.array(y)

print("✅ Feature shape:", X.shape)
print("✅ Labels shape:", y.shape)


  0%|          | 0/409 [00:00<?, ?it/s]/tmp/ipykernel_17/3607967621.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  0%|          | 1/409 [00:32<3:38:38, 32.15s/it]/tmp/ipykernel_17/3607967621.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  1%|          | 3/409 [00:33<51:18,  7.58s/it]  /tmp/ipykernel_17/3607967621.py:7: UserWarning: P

✅ Feature shape: (409, 32)
✅ Labels shape: (409,)


In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [8]:
xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)


In [9]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)


[0]	validation_0-rmse:0.76063
[1]	validation_0-rmse:0.75410
[2]	validation_0-rmse:0.74533
[3]	validation_0-rmse:0.74369
[4]	validation_0-rmse:0.73806
[5]	validation_0-rmse:0.72952
[6]	validation_0-rmse:0.72222
[7]	validation_0-rmse:0.71836
[8]	validation_0-rmse:0.71172
[9]	validation_0-rmse:0.70836
[10]	validation_0-rmse:0.69919
[11]	validation_0-rmse:0.69673
[12]	validation_0-rmse:0.69710
[13]	validation_0-rmse:0.69488
[14]	validation_0-rmse:0.69195
[15]	validation_0-rmse:0.68888
[16]	validation_0-rmse:0.68563
[17]	validation_0-rmse:0.68806
[18]	validation_0-rmse:0.68443
[19]	validation_0-rmse:0.68493
[20]	validation_0-rmse:0.68002
[21]	validation_0-rmse:0.67461
[22]	validation_0-rmse:0.67369
[23]	validation_0-rmse:0.67516
[24]	validation_0-rmse:0.67467
[25]	validation_0-rmse:0.67453
[26]	validation_0-rmse:0.67256
[27]	validation_0-rmse:0.67054
[28]	validation_0-rmse:0.67110
[29]	validation_0-rmse:0.67337
[30]	validation_0-rmse:0.67036
[31]	validation_0-rmse:0.67010
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=-1, num_parallel_tree=None, ...)

In [10]:
y_pred = xgb_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


RMSE: 0.6830
R² Score: 0.1543


In [11]:
single_audio_path = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/train/audio_173.wav"

single_feat = extract_audio_features(single_audio_path).reshape(1, -1)
prediction = xgb_model.predict(single_feat)

print("Predicted value:", prediction[0])


/tmp/ipykernel_17/3607967621.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Predicted value: 3.2602804


In [12]:
TEST_AUDIO_DIR = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test"
TEST_AUDIO_DIR

'/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test'

In [13]:
X_test_final = []
test_filenames = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    filename = row["filename"]

    if not filename.endswith(".wav"):
        filename_wav = filename + ".wav"
    else:
        filename_wav = filename

    audio_path = os.path.join(TEST_AUDIO_DIR, filename_wav)

    if not os.path.exists(audio_path):
        print(f"❌ Test file missing: {audio_path}")
        continue

    feats = extract_audio_features(audio_path)
    X_test_final.append(feats)
    test_filenames.append(row["filename"])


  3%|▎         | 6/197 [00:02<01:10,  2.71it/s]/tmp/ipykernel_17/3607967621.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  7%|▋         | 13/197 [00:04<01:08,  2.68it/s]/tmp/ipykernel_17/3607967621.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 17%|█▋        | 33/197 [00:12<00:59,  2.77it/s]/tmp/ipykernel_17/3607967621.py:7: UserWarn

In [14]:
X_test_final = np.array(X_test_final, dtype=np.float32)

print("✅ Test feature shape:", X_test_final.shape)


✅ Test feature shape: (197, 32)


In [15]:
test_predictions = xgb_model.predict(X_test_final)


In [16]:
test_predictions = np.clip(test_predictions, 0.0, 5.0)


In [17]:
submission = pd.DataFrame({
    "filename": test_filenames,
    "label": test_predictions
})

submission.head()


,filename,label
0,audio_141,2.422409
1,audio_114,2.634506
2,audio_17,2.597431
3,audio_76,3.342544
4,audio_156,3.538307


In [18]:
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv created successfully")


✅ submission.csv created successfully


In [19]:
print(submission.shape)          # must be (197, 2)
print(submission.isnull().sum()) # all zeros
print(submission["label"].min(), submission["label"].max())


(197, 2)
filename    0
label       0
dtype: int64
2.019648313522339 4.136220932006836
